# SweetEscape – Notebook 01: Modelltraining & Vergleich

In diesem Notebook trainieren wir drei Modelle auf dem vorverarbeiteten Datensatz (`data/processed/diabetes_fe.csv`)
und vergleichen sie mit identischen Splits und Metriken.

**Warum?**
So erhalten wir eine nachvollziehbare Modellentscheidung (Vergleichbarkeit) und speichern anschließend nur das finale Modell.

## Setup: Imports & Pfade

Wir laden die Libraries für Training, Evaluation und Speichern und definieren feste Pfade.

In [1]:
import pandas as pd
import joblib
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score

## Daten laden

Wir laden ausschließlich den processed Datensatz, damit Feature Engineering (Notebook 00) und Training getrennt bleiben.

In [2]:
PROJECT_ROOT = Path.cwd().parents[0]  # notebooks/ -> Projekt-Root
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "diabetes_fe.csv"
MODEL_PATH = PROJECT_ROOT / "models" / "diabetes_final_model.joblib"
TARGET = "Diabetes_012"

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head()

Shape: (253680, 29)


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,Age,Education,Income,inactive,cardio_risk_sum,low_fruits,low_veggies,lifestyle_risk_sum,poor_health,mental_physical_burden
0,0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,9.0,4.0,3.0,1,2.0,1,0,3.0,1,33.0
1,0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,7.0,6.0,1.0,0,0.0,1,1,3.0,0,0.0
2,0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,9.0,4.0,8.0,1,2.0,0,1,2.0,1,60.0
3,0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,11.0,3.0,6.0,0,1.0,0,0,0.0,0,0.0
4,0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,11.0,5.0,4.0,0,2.0,0,0,0.0,0,3.0


## Features und Zielvariable trennen

Wir trennen X und y und prüfen die Klassenverteilung (stark unausgeglichen).

In [3]:
y = df[TARGET].astype(int)
X = df.drop(columns=[TARGET])

display(y.value_counts().sort_index())
print("X:", X.shape, "y:", y.shape)

Diabetes_012
0    213703
1      4631
2     35346
Name: count, dtype: int64

X: (253680, 28) y: (253680,)


## Train/Test Split (stratifiziert)

Alle Modelle werden auf exakt demselben Split trainiert und getestet, damit die Metriken vergleichbar sind.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train-Verteilung:")
display(y_train.value_counts().sort_index())
print("Test-Verteilung:")
display(y_test.value_counts().sort_index())

Train-Verteilung:


Diabetes_012
0    170962
1      3705
2     28277
Name: count, dtype: int64

Test-Verteilung:


Diabetes_012
0    42741
1      926
2     7069
Name: count, dtype: int64

## Einheitliche Evaluation

Wir nutzen eine gemeinsame Funktion, die für jedes Modell die gleichen Ausgaben erzeugt
(Classification Report, Confusion Matrix, Macro-F1).

#### Evaluation: Warum nicht nur Accuracy?

Der Datensatz ist stark unausgeglichen (viele Fälle „kein Diabetes“, sehr wenige „Vorstufe“).
Ein Modell könnte deshalb eine hohe Accuracy erreichen, indem es fast immer nur die Mehrheitsklasse vorhersagt,
wäre aber für die Erkennung von Risikogruppen praktisch nutzlos.

Daher verwenden wir **Macro-F1** als primäres Vergleichsmaß:
Macro-F1 berechnet den F1-Score **für jede Klasse separat** und mittelt anschließend über alle Klassen,
sodass Minderheitsklassen (Vorstufe/Diabetes) gleichwertig berücksichtigt werden.
Zusätzlich betrachten wir die Confusion Matrix und insbesondere den Recall der Klassen 1 und 2.

In [5]:
def evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)

    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    macro_f1 = f1_score(y_test, y_pred, average="macro")
    print(f"Macro-F1: {macro_f1:.4f}")

    return {
        "name": name,
        "model": model,
        "macro_f1": macro_f1
    }

## Modell 1: Logistic Regression (Baseline)

Logistic Regression ist ein interpretierbares Baseline-Modell.
Mit `class_weight="balanced"` wird das Klassenungleichgewicht berücksichtigt.

In [6]:
logreg = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

logreg.fit(X_train, y_train)
res_logreg = evaluate_model("Logistic Regression (balanced)", logreg, X_test, y_test)


=== Logistic Regression (balanced) ===
              precision    recall  f1-score   support

           0       0.95      0.66      0.78     42741
           1       0.03      0.32      0.06       926
           2       0.35      0.59      0.44      7069

    accuracy                           0.64     50736
   macro avg       0.44      0.52      0.43     50736
weighted avg       0.85      0.64      0.72     50736

Confusion Matrix:
[[28248  7135  7358]
 [  236   299   391]
 [ 1193  1726  4150]]
Macro-F1: 0.4257


**Beobachtung:**
Die Logistic Regression erkennt die Mehrheitsklasse zuverlässig und zeigt eine moderate Erkennungsleistung
für Diabetes (Klasse 2). Die Vorstufe (Klasse 1) wird zwar teilweise erkannt, weist jedoch einen niedrigen Recall auf,
was die Schwierigkeit dieser Klasse widerspiegelt.

## Modell 2: Random Forest (nicht-linear)

Random Forest kann nicht-lineare Zusammenhänge lernen.
Wir nutzen `class_weight="balanced_subsample"` als einfache Gewichtung gegen das Ungleichgewicht.

In [7]:
rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced_subsample"
)

rf.fit(X_train, y_train)
res_rf = evaluate_model("Random Forest (balanced_subsample)", rf, X_test, y_test)


=== Random Forest (balanced_subsample) ===
              precision    recall  f1-score   support

           0       0.86      0.97      0.91     42741
           1       0.00      0.00      0.00       926
           2       0.47      0.16      0.24      7069

    accuracy                           0.84     50736
   macro avg       0.44      0.38      0.38     50736
weighted avg       0.79      0.84      0.80     50736

Confusion Matrix:
[[41370   126  1245]
 [  842     0    84]
 [ 5894    15  1160]]
Macro-F1: 0.3845


**Beobachtung:**
Der Random Forest erzielt eine hohe Accuracy durch sehr gute Erkennung der Mehrheitsklasse,
versagt jedoch nahezu vollständig bei der Erkennung der Vorstufe (Recall ≈ 0).
Dies führt trotz hoher Gesamtgenauigkeit zu einem deutlich schlechteren Macro-F1-Wert.

## Modell 3: HistGradientBoosting (Boosting)

Boosting-Modelle sind oft sehr stark bei tabellarischen Daten.
Da `HistGradientBoostingClassifier` kein `class_weight` hat, nutzen wir sample weights aus der Klassenverteilung.

In [8]:
# sample_weight berechnen: seltene Klassen bekommen höheres Gewicht
class_counts = y_train.value_counts().to_dict()
max_count = max(class_counts.values())
weights = y_train.map(lambda c: max_count / class_counts[c]).values

hgb = HistGradientBoostingClassifier(
    random_state=42,
    max_depth=None,
    learning_rate=0.1,
    max_iter=300
)

hgb.fit(X_train, y_train, sample_weight=weights)
res_hgb = evaluate_model("HistGradientBoosting (sample_weight)", hgb, X_test, y_test)


=== HistGradientBoosting (sample_weight) ===
              precision    recall  f1-score   support

           0       0.96      0.63      0.76     42741
           1       0.03      0.31      0.05       926
           2       0.35      0.62      0.45      7069

    accuracy                           0.62     50736
   macro avg       0.44      0.52      0.42     50736
weighted avg       0.85      0.62      0.70     50736

Confusion Matrix:
[[26850  8118  7773]
 [  227   289   410]
 [ 1026  1652  4391]]
Macro-F1: 0.4192


**Beobachtung:**
Das Gradient-Boosting-Modell zeigt eine ähnliche Performance wie die Logistic Regression
bei der Erkennung von Diabetes (Klasse 2), erkennt die Vorstufe jedoch ebenfalls nur eingeschränkt.
Insgesamt ergibt sich eine vergleichbare, aber leicht schlechtere Balance über alle Klassen.

## Vergleich & Modellentscheidung

Wir vergleichen die Modelle anhand von Macro-F1 (wichtig bei Klassenungleichgewicht) und betrachten zusätzlich
Recall der Minderheitsklassen (Klasse 1 und 2) im Classification Report.

Am Ende wählen wir ein finales Modell für die Web-App und speichern nur dieses.

In [9]:
results = pd.DataFrame([
    {"Model": res_logreg["name"], "Macro-F1": res_logreg["macro_f1"]},
    {"Model": res_rf["name"], "Macro-F1": res_rf["macro_f1"]},
    {"Model": res_hgb["name"], "Macro-F1": res_hgb["macro_f1"]},
]).sort_values("Macro-F1", ascending=False)

results

,Model,Macro-F1
0,Logistic Regression (balanced),0.425669
2,HistGradientBoosting (sample_weight),0.419234
1,Random Forest (balanced_subsample),0.384497


## Finales Modell speichern

Wir speichern nur das ausgewählte finale Modell als `.joblib`, damit die Web-App es laden kann.
Die Wahl begründen wir anhand der Vergleichsmetriken (insb. Macro-F1 und Minderheitsklassen-Recall).

In [10]:
# Entscheidung: standardmäßig bestes nach Macro-F1
best = max([res_logreg, res_rf, res_hgb], key=lambda r: r["macro_f1"])
best_model = best["model"]

MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(best_model, MODEL_PATH)

print("✅ Finales Modell:", best["name"])
print("✅ Gespeichert unter:", MODEL_PATH)

✅ Finales Modell: Logistic Regression (balanced)
✅ Gespeichert unter: C:\Users\Jan\DataspellProjects\SweetEscape\models\diabetes_final_model.joblib


## Begründung

Das finale Modell wurde auf Basis eines fairen Vergleichs (gleicher Train/Test-Split) ausgewählt.
Bei starkem Klassenungleichgewicht ist Macro-F1 aussagekräftiger als reine Accuracy.
Zur Interpretation der Ergebnisse werden zusätzlich der Recall der Minderheitsklassen sowie die Confusion Matrix betrachtet, um kritische Fehlklassifikationen sichtbar zu machen.

**Warum Logistic Regression?**
Während komplexere Modelle teilweise eine höhere Accuracy erreichen, liefert die Logistic Regression die ausgewogenste Leistung über alle Klassen hinweg, was sich im höchsten Macro-F1-Wert widerspiegelt.